In [31]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import keras
from keras import layers
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
import random as random


trainset_1 = pd.read_csv('EEG_mouse_data_1.csv')
trainset_2 = pd.read_csv('EEG_mouse_data_2.csv')
testset_1 = pd.read_csv('EEG_mouse_data_test.csv')

In [32]:
# visualisation des données par classe
def show_boxplots(df):
    fig, axs = plt.subplots(20, 5, figsize=(25, 120))
    axs = axs.flatten()

    for i, col in enumerate(df.columns[1:]):
        sns.boxplot(x='state', y=col, data=df, ax=axs[i])
    
    fig.suptitle('Boxplot of the amplitudes', fontsize=14)
    fig.tight_layout(rect=[0, 0.03, 1, 0.98])


In [33]:
# normlisation des features
from sklearn.preprocessing import StandardScaler
import pandas as pd

def normalize(df):
    headers = df.columns
    scaler = StandardScaler()
    
    numerical_columns = df.select_dtypes(include=['int', 'float']).columns
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
    normalized_data = pd.DataFrame(df, columns=headers[:-1])

    return normalized_data

In [34]:
# extraction des colonnes
# envisager de faire la séléction des features ici
def extract_features(data, n):
    return data.iloc[:,:n+1]


In [35]:
def create_model():
  # 2 inputs
  # 2 hidden nodes
  # 1 output

  mlp = keras.Sequential([
      layers.Input(shape=(25,)),
      layers.Dense(5, activation="tanh"), # Try different numbers of hidden neurons here (e.g. 2, 4, 8, 32, 128)
      layers.Dense(1, activation="tanh"),
  ])

  # Experiment with hyperparameters here:
  # momentum: [0, 0.8, 0.9, 0.99]
  # learning_rate: [0.1, 0.01, 0.001, 0.0001]
  mlp.compile(
      optimizer=keras.optimizers.SGD(learning_rate=0.05, momentum=0.99),
      loss="mse",
  )

  return mlp
mlp = create_model()
mlp.summary()


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_14 (Dense)                     │ (None, 5)                   │             130 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 1)                   │               6 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 136 (544.00 B)

 Trainable params: 136 (544.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import keras
from keras import layers
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
import random as random


# Programme principal de traitement du train set #1
normal_trainset_1 = extract_features(trainset_1, 26)
normal_trainset_1 = normalize(normal_trainset_1)
normal_trainset_1 = normal_trainset_1.sample(frac=1, random_state=42)

# splitting en 3 folds
keras.utils.set_random_seed(123)
kf = KFold(n_splits=3, shuffle=True)

y = normal_trainset_1['state'].map({'w': 1, 'r': 0, 'n': 0})
normal_trainset_1.drop(columns=['state'], inplace=True)

history_list = []
trained_mlp = []

for fold, (train_index, test_index) in enumerate(kf.split(normal_trainset_1)):
    
    model = create_model()
    
    history = model.fit(x=normal_trainset_1.iloc[train_index], y=y.iloc[train_index], 
                       validation_data=(normal_trainset_1.iloc[test_index], y.iloc[test_index]),
                       epochs=500)
    
    history_list.append(history)
    trained_mlp.append(model)


Epoch 1/500
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2107 - val_loss: 0.1009
Epoch 2/500
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1231 - val_loss: 0.1417
Epoch 3/500
423/423 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1690 - val_loss: 0.2122
Epoch 4/500
423/423 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1624 - val_loss: 0.1068
Epoch 5/500
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0978 - val_loss: 0.3239
Epoch 6/500
423/423 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1828 - val_loss: 0.1091
Epoch 7/500
423/423 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1079 - val_loss: 0.0974
Epoch 8/500
423/423 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1044 - val_loss: 0.1948
Epoch 9/500
423/423 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1702 - val_loss: 0.2927
Epoch 10/500
423/423 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2304 - val_loss: 0.1981
Epoch 11/500
423/423 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1253 - val_loss: 0.1333
Epoch 12/500
423/423 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import matplotlib.pyplot as pl
train_losses = np.array([history.history['loss'] for history in history_list])
val_losses = np.array([history.history['val_loss'] for history in history_list])

# Calculate mean and standard deviation for training and validation losses
mean_train_loss = np.mean(train_losses, axis=0)
std_train_loss = np.std(train_losses, axis=0)
mean_val_loss = np.mean(val_losses, axis=0)
std_val_loss = np.std(val_losses, axis=0)

# Plot mean and standard deviation for training loss
pl.plot(mean_train_loss, label='Training Loss (Mean)')
pl.fill_between(range(len(mean_train_loss)), mean_train_loss - std_train_loss, mean_train_loss + std_train_loss, alpha=0.3, label='Training Loss (Std)')

# Plot mean and standard deviation for validation loss
pl.plot(mean_val_loss, label='Validation Loss (Mean)')
pl.fill_between(range(len(mean_val_loss)), mean_val_loss - std_val_loss, mean_val_loss + std_val_loss, alpha=0.3, label='Validation Loss (Std)')

# Add labels and legend
pl.xlabel('Epochs')
pl.ylabel('Loss')
pl.legend()

# Display the plot
pl.show()

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, f1_score
import seaborn as sns

def plot_confusion_matrix(confusion_matrix, title):
    # Plot confusion matrix
    pl.figure(figsize=(8, 6))
    sns.heatmap(confusion_matrix.astype(int), annot=True, fmt="d", cmap="Blues", cbar=False,
                xticklabels=["w", "r/nr"], yticklabels=["w", "r/n"])
    pl.title(title)
    pl.xlabel('Predicted')
    pl.ylabel('True')
    pl.show()

f1_scores = []
mean_confusion_matrix = np.zeros((2, 2))

for i, (train_index, test_index) in enumerate(kf.split(normal_trainset_1)):
    # Evaluate the trained model on the test fold
    predictions = (trained_mlp[i].predict(normal_trainset_1.iloc[test_index]) > 0).astype(int)
    true_labels = (y.iloc[test_index] > 0).astype(int)

    # Compute confusion matrix
    cm = confusion_matrix(true_labels, predictions)
    mean_confusion_matrix += confusion_matrix(true_labels, predictions)

    # Compute confusion matrix and plot
    plot_confusion_matrix(cm, f'Confusion Matrix - Fold {i + 1}')

    # Compute F1 score
    f1 = f1_score(true_labels, predictions)
    f1_scores.append(f1)
    print(f"F1 Score - Fold {i + 1}: {f1}")

# Plot mean confusion matrix
plot_confusion_matrix(mean_confusion_matrix, 'Global confusion matrix')

# Calculate and display the mean F1 score across all folds
mean_f1_score = np.mean(f1_scores)
print(f"Mean F1 Score across all folds: {mean_f1_score}")